In [1]:
import pandas as pd
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim
from unidecode import unidecode
import numpy as np
from shapely.geometry import Point, Polygon
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [4]:
#risk2023.to_file("C:\\Users\\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\1. Mapas\Metodo_1\\areas\\municipiosrisk.shp")

In [8]:
risk2023 = gpd.read_file("C:\Users\puria\Documents\articulo\Mapa_Migracion_Irregular_Mexico\project\1.Mapas\areas\\municipiosrisk1.shp")


In [9]:
risk2023.head(10)

,nommun,cvemun,nomedo,area,R1,R2,R3,R4,R4pt,geometry
0,"Aguascalientes, Aguascalientes",1001,Aguascalientes,1168.762384,0.360084,0.372695,0.117251,0.662296,0.000458,"POLYGON ((-11370403.590 2520317.662, -11368883..."
1,"Asientos, Aguascalientes",1002,Aguascalientes,547.762077,0.019696,0.011381,0.017855,0.745202,0.000515,"POLYGON ((-11368305.122 2546014.561, -11367916..."
2,"Calvillo, Aguascalientes",1003,Aguascalientes,931.300088,0.019349,0.013107,0.013835,0.708078,0.000489,"POLYGON ((-11444356.824 2510725.426, -11444212..."
3,"Cosio, Aguascalientes",1004,Aguascalientes,128.907513,0.012770,0.010345,0.012474,0.704691,0.000487,"POLYGON ((-11387740.145 2566722.259, -11387556..."
4,"El Llano, Aguascalientes",1010,Aguascalientes,500.861359,0.012770,0.012417,0.011113,0.710027,0.000491,"POLYGON ((-11365483.996 2514328.561, -11364488..."
5,"Jesus Maria, Aguascalientes",1005,Aguascalientes,499.207990,0.041511,0.013453,0.026453,0.697694,0.000482,"POLYGON ((-11400837.207 2516051.901, -11399144..."
6,"Pabellon de Arteaga, Aguascalientes",1006,Aguascalientes,199.342229,0.021427,0.012762,0.015381,0.702479,0.000486,"POLYGON ((-11377087.802 2530916.387, -11377135..."
7,"Rincon de Romos, Aguascalientes",1007,Aguascalientes,372.937842,0.023158,0.012762,0.014453,0.742308,0.000513,"POLYGON ((-11382606.455 2556483.698, -11382217..."
8,"San Francisco de los Romo, Aguascalientes",1011,Aguascalientes,134.074435,0.029738,0.013107,0.020206,0.699417,0.000483,"POLYGON ((-11375973.967 2519429.178, -11375646..."
9,"San Jose de Gracia, Aguascalientes",1008,Aguascalientes,856.214195,0.012078,0.011036,0.010680,0.748600,0.000517,"POLYGON ((-11423205.269 2546540.540, -11422408..."


In [87]:
# Seleccionar las columnas que deseas integrar y renombrarlas
columns_to_merge = ['cvemunic', 'risk2_trato_general_normalizado_ajustado', 'risk2_abusos_autoridades_normalizado_ajustado', 'risk2_estado_instalaciones_normalizado_ajustado']

# Crear un diccionario para renombrar las columnas
columns_renamed = {
    'risk2_trato_general_normalizado_ajustado': 'risk2_trato_general',
    'risk2_abusos_autoridades_normalizado_ajustado': 'risk2_abusos_autoridades',
    'risk2_estado_instalaciones_normalizado_ajustado': 'risk2_estado_instalaciones'
}

# Seleccionar y renombrar las columnas en valores_emif
valores_emif_selected = valores_emif[columns_to_merge].rename(columns=columns_renamed)

# Hacer el merge de las dataframes usando la columna 'cvemunic'
risk2023 = risk2023.merge(valores_emif_selected, on='cvemunic', how='left')

In [92]:
# Inicializar el escalador
scaler = MinMaxScaler(feature_range=(1, 100))

# Normalizar los valores de la columna 'corrupcion'
risk2023['risk2_delservpub'] = scaler.fit_transform(risk2023[['corrupcion']])

In [96]:
# Asegúrate de que las columnas están en el DataFrame
columns_to_sum = ['risk2_trato_general', 'risk2_abusos_autoridades', 'risk2_estado_instalaciones', 'risk2_delservpub']

# Verifica si todas las columnas existen en el DataFrame
for col in columns_to_sum:
    if col not in risk2023.columns:
        print(f"Warning: Column '{col}' not found in the DataFrame.")
        
# Sumar las columnas
risk2023['total_vio_inst'] = risk2023[columns_to_sum].sum(axis=1)

# Inicializar el escalador
scaler = MinMaxScaler(feature_range=(1, 100))

# Normalizar los valores de la columna 'total_vio_inst'
risk2023['vio_inst'] = scaler.fit_transform(risk2023[['total_vio_inst']])

# Eliminar la columna 'total_vio_inst' si ya no es necesaria
risk2023.drop(columns=['total_vio_inst'], inplace=True)


### Normalizar todos los stats

## Stat 1: Violencia e Inseguridad Mujeres, Familias y Ninos.

From databse del secretariado ejectuvido del sistema nacional de seguridad publica.
es el numero de eventos en 2023, por division municipal de: 'Feminicidio','Tráfico de menores','La libertad y la seguridad sexual'; Acoso Sexucal, Abuso Sexual, Hostigamiento Sexual, Violacion Simple, Violacion Equiparada. Y 'La sociedad'; ,;, . 

In [7]:
# Normalizar la columna 'Violencia' en una escala de 1 a 100
min_val = risk2023['Violencia'].min()
max_val = risk2023['Violencia'].max()

risk2023['Violencia_normalizada'] = (risk2023['Violencia'] - min_val) / (max_val - min_val) * 99 + 1


## Stat 2:

-'Delitos cometidos por servidores públicos' fromo databe secretariado ejectuvido del sistema nacional de seguridad publica. return 'Corrupción Institucional'

-emifsur - DEVUELTOS POR AUTORIDADES 
MEXICANA (contabilizacion abusos de autoridad)
S

In [31]:
emifdevgua1 = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\Guate-Julio-Septiembre 2022.csv")
emifdevgua2 = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\Guate-Octubre-Noviembre 2022.csv")
emifdevhon1 = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\hon-Julio-Septiembre 2022.csv")
emifdevhon2 = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\hon-Octubre-Noviembre 2022.csv")
emifdevsal1 = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\sal-Julio-Septiembre 2022.csv")
emifdevsal2 = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\sal-Octubre-Noviembre 2022.csv")


In [35]:
# Función para obtener columnas que comienzan con 'p34'
def obtener_columnas_p34(df):
    return [col for col in df.columns if col.startswith('p34')]

# Obtener columnas que comienzan con 'p34' para cada dataframe
columnas_p34_gua1 = obtener_columnas_p34(emifdevgua1)
columnas_p34_gua2 = obtener_columnas_p34(emifdevgua2)
columnas_p34_hon1 = obtener_columnas_p34(emifdevhon1)
columnas_p34_hon2 = obtener_columnas_p34(emifdevhon2)
columnas_p34_sal1 = obtener_columnas_p34(emifdevsal1)
columnas_p34_sal2 = obtener_columnas_p34(emifdevsal2)

# Contar las columnas que comienzan con 'p34' para cada dataframe
conteo_columnas_p34 = {
    'emifdevgua1': len(columnas_p34_gua1),
    'emifdevgua2': len(columnas_p34_gua2),
    'emifdevhon1': len(columnas_p34_hon1),
    'emifdevhon2': len(columnas_p34_hon2),
    'emifdevsal1': len(columnas_p34_sal1),
    'emifdevsal2': len(columnas_p34_sal2)
}

# Mostrar resultados
print("Columnas que comienzan con 'p34' y su conteo en cada dataframe:")
for df_name, count in conteo_columnas_p34.items():
    print(f"{df_name}: {count} columnas")
    if df_name == 'emifdevgua1':
        print(f"Columnas: {columnas_p34_gua1}")
    elif df_name == 'emifdevgua2':
        print(f"Columnas: {columnas_p34_gua2}")
    elif df_name == 'emifdevhon1':
        print(f"Columnas: {columnas_p34_hon1}")
    elif df_name == 'emifdevhon2':
        print(f"Columnas: {columnas_p34_hon2}")
    elif df_name == 'emifdevsal1':
        print(f"Columnas: {columnas_p34_sal1}")
    elif df_name == 'emifdevsal2':
        print(f"Columnas: {columnas_p34_sal2}")
    print()

# Comparar las listas de columnas para ver diferencias
columnas_set_gua1 = set(columnas_p34_gua1)
columnas_set_gua2 = set(columnas_p34_gua2)
columnas_set_hon1 = set(columnas_p34_hon1)
columnas_set_hon2 = set(columnas_p34_hon2)
columnas_set_sal1 = set(columnas_p34_sal1)
columnas_set_sal2 = set(columnas_p34_sal2)

diferencias_gua = columnas_set_gua1.symmetric_difference(columnas_set_gua2)
diferencias_hon = columnas_set_hon1.symmetric_difference(columnas_set_hon2)
diferencias_sal = columnas_set_sal1.symmetric_difference(columnas_set_sal2)

print(f"Diferencias entre Guate-Julio-Septiembre 2022 y Guate-Octubre-Noviembre 2022: {diferencias_gua}")
print(f"Diferencias entre hon-Julio-Septiembre 2022 y hon-Octubre-Noviembre 2022: {diferencias_hon}")
print(f"Diferencias entre sal-Julio-Septiembre 2022 y sal-Octubre-Noviembre 2022: {diferencias_sal}")

Columnas que comienzan con 'p34' y su conteo en cada dataframe:
emifdevgua1: 51 columnas
Columnas: ['p34e', 'p34m', 'p34l', 'p34u', 'p34_1', 'p34_1_1', 'p34_2', 'p34_2_1_1', 'p34_2_1_2', 'p34_2_1_3', 'p34_2_1_4', 'p34_2_1_5', 'p34_2_1_6', 'p34_2_2', 'p34_2_3_1', 'p34_2_3_2', 'p34_2_3_3', 'p34_3_0_1', 'p34_3_0_2', 'p34_3_0_3', 'p34_3_0_4', 'p34_3_1_1', 'p34_3_1_2', 'p34_3_1_3', 'p34_3_1_4', 'p34_4c', 'p34_4t', 'p34_5_1', 'p34_5_2', 'p34_5_3', 'p34_5_4', 'p34_5_5', 'p34_5_6', 'p34_5_7', 'p34_6', 'p34_7', 'p34_8', 'p34_9', 'p34_10_1', 'p34_10_2', 'p34_10_3', 'p34_10_4', 'p34_10_5', 'p34_10_6', 'p34_10_7', 'p34_10_8', 'p34_11', 'p34_12', 'p34_12_1', 'p34_13', 'p34_14']

emifdevgua2: 51 columnas
Columnas: ['p34e', 'p34m', 'p34l', 'p34u', 'p34_1', 'p34_1_1', 'p34_2', 'p34_2_1_1', 'p34_2_1_2', 'p34_2_1_3', 'p34_2_1_4', 'p34_2_1_5', 'p34_2_1_6', 'p34_2_2', 'p34_2_3_1', 'p34_2_3_2', 'p34_2_3_3', 'p34_3_0_1', 'p34_3_0_2', 'p34_3_0_3', 'p34_3_0_4', 'p34_3_1_1', 'p34_3_1_2', 'p34_3_1_3', 'p34_3_1_

In [33]:
emifdevhon1

,folio,unidad,trim,ciudad,punto,turno,p1,p2,p3a,p3n,...,p52,p52_1,p52_2,p52_3,nivel,oficio_mx,filtro,estrato,upm,ponfin3
0,709,12209,3,1,1,1,2,36,6,1,...,1,1,-44,-44,2,0,1,1011,1011012209,296.639812
1,1708,12208,3,1,1,1,2,25,3,3,...,1,2,2,1,4,0,1,1011,1011012208,127.281621
2,2208,12208,3,1,1,1,1,27,2,1,...,1,2,1,1,1,0,1,1011,1011012208,195.555395
3,3808,22208,3,1,1,1,1,24,3,2,...,-40,-40,-40,-40,3,0,1,1011,1011022208,316.767417
4,5708,32208,3,1,1,1,1,27,5,1,...,1,2,1,2,1,0,1,1011,1011032208,122.828182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,61308,242208,3,1,1,1,1,22,6,1,...,1,2,1,1,2,0,1,1011,1011242208,256.969486
70,66308,292208,3,1,1,1,1,37,3,1,...,1,1,-44,-44,1,7,1,1011,1011292208,127.676663
71,66608,292208,3,1,1,1,1,19,3,2,...,1,1,-44,-44,3,0,1,1011,1011292208,127.676663
72,66908,292208,3,1,1,1,1,19,0,0,...,-39,-39,-39,-39,0,0,1,1011,1011292208,127.676663


In [45]:
# Concatenar los dataframes
emifdev_combined = pd.concat([emifdevgua1, emifdevgua2, emifdevhon1, emifdevhon2, emifdevsal1, emifdevsal2])

# Filtrar las columnas que comienzan con 'p34'
columnas_p34 = [col for col in emifdev_combined.columns if col.startswith('p34')]
 = emifdev_combined[columnas_p34]

# Verificar el número de filas y columnas en el dataframe filtrado
print(f"Número de filas en el dataframe combinado: {emifdev_combined.shape[0]}")
print(f"Número de columnas en el dataframe combinado: {emifdev_combined.shape[1]}")
print(f"Número de filas en el dataframe filtrado: {emifdev_filtrado.shape[0]}")
print(f"Número de columnas en el dataframe filtrado: {emifdev_filtrado.shape[1]}")


Número de filas en el dataframe combinado: 2218
Número de columnas en el dataframe combinado: 241
Número de filas en el dataframe filtrado: 2218
Número de columnas en el dataframe filtrado: 51


In [59]:
#emifdev_filtrado.to_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\vio_inst_emifsurdev.csv")

In [60]:
emifdev_filtrado = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\vio_inst_emifsurdev.csv")

In [43]:
dicc = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\DMXGT_Valores_2022.csv")

In [44]:
dicc

,Variable,Valor,Etiqueta
0,trim,3,Jul - Sep 2022
1,punto,97,No sabe
2,punto,98,No responde
3,punto,99,No especificado
4,turno,1,Único
...,...,...,...
19716,oficio_mx,6,Domésticos
19717,oficio_mx,7,Servicios
19718,oficio_mx,8,Profesionistas
19719,oficio_mx,99,No especificado


In [55]:
# Crear diccionarios de mapeo para cada columna de interés
mapeo_p34e = dicc[dicc['Variable'] == 'p34e'].set_index('Valor')['Etiqueta'].to_dict()
mapeo_p34m = dicc[dicc['Variable'] == 'p34m'].set_index('Valor')['Etiqueta'].to_dict()
mapeo_p34l = dicc[dicc['Variable'] == 'p34l'].set_index('Valor')['Etiqueta'].to_dict()

# Crear una nueva columna 'cvemunic' con los valores originales de 'p34m'
emifdev_filtrado['cvemunic'] = emifdev_filtrado['p34m']

# Aplicar los diccionarios de mapeo a las columnas correspondientes en emifdev_filtrado
emifdev_filtrado['p34e'] = emifdev_filtrado['p34e'].map(mapeo_p34e)
emifdev_filtrado['p34m'] = emifdev_filtrado['p34m'].map(mapeo_p34m)
emifdev_filtrado['p34l'] = emifdev_filtrado['p34l'].map(mapeo_p34l)


In [57]:
# Realizar el join para agregar la columna 'municipio_' a emifdev_filtrado basado en 'cvemunic'
emifdev_filtrado = emifdev_filtrado.merge(risk2023[['cvemunic', 'municipio_']], on='cvemunic', how='left')


In [63]:
emifdev_filtrado.head()

,municipio_,cvemunic,p34e,p34m,p34l,p34u,p34_1,p34_1_1,p34_2,p34_2_1_1,...,p34_10_5,p34_10_6,p34_10_7,p34_10_8,p34_11,p34_12,p34_12_1,p34_13,p34_14,Unnamed: 53
0,"Benito Juarez, Ciudad de Mexico",9014,Distrito Federal,Benito Juárez,Benito Juárez (Distrito Federal),2,3,2,2,1,...,1,1,1,2,1,2,-32,2,-33,NaN
1,"Cuernavaca, Morelos",17007,Morelos,Cuernavaca,Cuernavaca (Morelos),2,3,2,2,1,...,1,1,1,2,1,2,-32,2,-33,NaN
2,"Cuauhtemoc, Ciudad de Mexico",9015,Distrito Federal,Cuauhtémoc,Cuauhtémoc (Distrito Federal),2,4,1,3,2,...,2,2,2,2,1,2,-32,2,-33,NaN
3,"Benito Juarez, Ciudad de Mexico",9014,Distrito Federal,Benito Juárez,Benito Juárez (Distrito Federal),2,4,1,2,2,...,2,2,2,2,1,2,-32,2,-33,NaN
4,"Puebla, Puebla",21114,Puebla,Puebla,Heróica Puebla de Zaragoza (Puebla),2,8,1,3,1,...,2,2,2,2,1,2,-32,2,-33,NaN


### scores stat 2

In [64]:
# Crear un dataframe con el conteo de apariciones de cada 'cvemunic'
valores_emif = emifdev_filtrado.groupby(['cvemunic', 'municipio_']).size().reset_index(name='frec_app')

# Crear las columnas 'p34_2:4frec' y 'p34_2:5frec'
valores_emif['p34_2:4frec'] = emifdev_filtrado[emifdev_filtrado['p34_2'] == 4].groupby('cvemunic').size().reindex(valores_emif['cvemunic'], fill_value=0).values
valores_emif['p34_2:5frec'] = emifdev_filtrado[emifdev_filtrado['p34_2'] == 5].groupby('cvemunic').size().reindex(valores_emif['cvemunic'], fill_value=0).values

# Calcular 'p34_2_score'
valores_emif['p34_2_score'] = valores_emif['p34_2:4frec'] * 30 + valores_emif['p34_2:5frec'] * 70

# Crear las columnas 'p34_3_0_1frec', 'p34_3_0_2frec', 'p34_3_0_3frec', 'p34_3_0_4frec'
for col in ['p34_3_0_1', 'p34_3_0_2', 'p34_3_0_3', 'p34_3_0_4']:
    valores_emif[col + 'frec'] = emifdev_filtrado[emifdev_filtrado[col] == 1].groupby('cvemunic').size().reindex(valores_emif['cvemunic'], fill_value=0).values

# Calcular 'p34_3_score'
valores_emif['p34_3_score'] = valores_emif[['p34_3_0_1frec', 'p34_3_0_2frec', 'p34_3_0_3frec', 'p34_3_0_4frec']].sum(axis=1) * 25

# Crear las columnas 'p34_5_1frec', 'p34_5_2frec', 'p34_5_3frec', 'p34_5_4frec', 'p34_5_5frec', 'p34_5_6frec', 'p34_5_7frec'
for col in ['p34_5_1', 'p34_5_2', 'p34_5_3', 'p34_5_4', 'p34_5_5', 'p34_5_6', 'p34_5_7']:
    valores_emif[col + 'frec'] = emifdev_filtrado[emifdev_filtrado[col] == 2].groupby('cvemunic').size().reindex(valores_emif['cvemunic'], fill_value=0).values

# Calcular 'p34_5_score'
valores_emif['p34_5_score'] = valores_emif[['p34_5_1frec', 'p34_5_2frec', 'p34_5_3frec', 'p34_5_4frec', 'p34_5_5frec', 'p34_5_6frec', 'p34_5_7frec']].sum(axis=1) * 14.28


In [94]:
#valores_emif.to_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\vio_inst_emifsurdev_scores.csv")

In [80]:
valores_emif = pd.read_csv("C:\\Users\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\\EMIF-Devueltos\\vio_inst_emifsurdev_scores.csv")

In [72]:
# Ordenar el DataFrame de mayor a menor basado en la columna 'frec_app'
valores_emif_sorted = valores_emif.sort_values(by='p34_2:4frec', ascending=False)


In [95]:
valores_emif.head(10)

,Unnamed: 0,cvemunic,municipio_,frec_app,p34_2:4frec,p34_2:5frec,p34_2_score,p34_3_0_1frec,p34_3_0_2frec,p34_3_0_3frec,...,risk2_estado_instalaciones,risk2_trato_general_normalizado,risk2_abusos_autoridades_normalizado,risk2_estado_instalaciones_normalizado,risk2_trato_general_ajustado,risk2_abusos_autoridades_ajustado,risk2_estado_instalaciones_ajustado,risk2_trato_general_normalizado_ajustado,risk2_abusos_autoridades_normalizado_ajustado,risk2_estado_instalaciones_normalizado_ajustado
0,0,1001,"Aguascalientes, Aguascalientes",10,0,1,70,1,1,2,...,51.408,10.000000,20.000000,51.428571,29.192267,41.703239,214.388012,28.187135,13.268622,11.241027
1,1,2001,"Ensenada, Baja California",3,0,1,70,1,1,1,...,71.400,33.333333,50.000000,71.428571,50.494326,54.101064,154.512639,48.755733,17.213209,8.101576
2,2,2002,"Mexicali, Baja California",10,0,0,0,3,2,0,...,82.824,0.000000,30.000000,82.857143,0.000000,62.554859,345.402908,0.000000,19.902933,18.110543
3,3,2004,"Tijuana, Baja California",15,4,0,120,6,1,1,...,59.024,11.428571,26.666667,59.047619,43.280851,72.134752,319.326120,41.790628,22.950946,16.743257
4,4,4001,"Calkini, Campeche",1,0,0,0,0,0,0,...,57.120,0.000000,0.000000,57.142857,0.000000,0.000000,82.406741,0.000000,0.000000,4.320840
5,5,4002,"Campeche, Campeche",3,0,0,0,1,1,0,...,33.320,0.000000,33.333333,33.333333,0.000000,36.067376,72.105898,0.000000,11.475473,3.780735
6,6,4004,"Champoton, Campeche",1,0,0,0,0,0,0,...,57.120,0.000000,0.000000,57.142857,0.000000,0.000000,82.406741,0.000000,0.000000,4.320840
7,7,4009,"Escarcega, Campeche",3,0,0,0,0,0,0,...,61.880,0.000000,0.000000,61.904762,0.000000,0.000000,133.910954,0.000000,0.000000,7.021366
8,8,5002,"Acuna, Coahuila de Zaragoza",2,0,0,0,0,0,0,...,49.980,0.000000,0.000000,50.000000,0.000000,0.000000,90.987513,0.000000,0.000000,4.770757
9,9,5003,"Allende, Coahuila de Zaragoza",1,0,0,0,0,0,0,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [82]:
import pandas as pd

# Supongamos que el dataframe valores_emif ya está cargado y contiene las columnas necesarias

# Calcular el score promedio para cada indicador
valores_emif['risk2_trato_general'] = valores_emif['p34_2_score'] / valores_emif['frec_app']
valores_emif['risk2_abusos_autoridades'] = valores_emif['p34_3_score'] / valores_emif['frec_app']
valores_emif['risk2_estado_instalaciones'] = valores_emif['p34_5_score'] / valores_emif['frec_app']

# Normalizar cada score en una escala de 0 a 100 utilizando el método min-max
def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min()) * 100

valores_emif['risk2_trato_general_normalizado'] = min_max_scaling(valores_emif['risk2_trato_general'])
valores_emif['risk2_abusos_autoridades_normalizado'] = min_max_scaling(valores_emif['risk2_abusos_autoridades'])
valores_emif['risk2_estado_instalaciones_normalizado'] = min_max_scaling(valores_emif['risk2_estado_instalaciones'])


In [83]:
# Calcular el score promedio ajustado utilizando el logaritmo de frec_app
valores_emif['risk2_trato_general_ajustado'] = valores_emif['p34_2_score'] / np.log1p(valores_emif['frec_app'])
valores_emif['risk2_abusos_autoridades_ajustado'] = valores_emif['p34_3_score'] / np.log1p(valores_emif['frec_app'])
valores_emif['risk2_estado_instalaciones_ajustado'] = valores_emif['p34_5_score'] / np.log1p(valores_emif['frec_app'])

# Normalizar cada score ajustado en una escala de 0 a 100 utilizando el método min-max
valores_emif['risk2_trato_general_normalizado_ajustado'] = min_max_scaling(valores_emif['risk2_trato_general_ajustado'])
valores_emif['risk2_abusos_autoridades_normalizado_ajustado'] = min_max_scaling(valores_emif['risk2_abusos_autoridades_ajustado'])
valores_emif['risk2_estado_instalaciones_normalizado_ajustado'] = min_max_scaling(valores_emif['risk2_estado_instalaciones_ajustado'])



In [27]:
# Filtrar las columnas que comienzan con 'p34_3'
columnas_p34_3 = [col for col in emifdev1_filtrado.columns if col.startswith('p34_3')]

# Crear una máscara booleana para filtrar las filas que tienen el valor '1' en cualquiera de las columnas p34_3
mask = emifdev1_filtrado[columnas_p34_3].eq(1).any(axis=1)

# Filtrar el dataframe utilizando la máscara
emifdev1_filtrado_1 = emifdev1_filtrado[mask]

# Verificar el número de filas en el dataframe original y el filtrado
print(f"Número de filas en el dataframe original: {emifdev1_filtrado.shape[0]}")
print(f"Número de filas en el dataframe filtrado: {emifdev1_filtrado_1.shape[0]}")


Número de filas en el dataframe original: 796
Número de filas en el dataframe filtrado: 147


In [28]:
emifdev1_filtrado_1

,p34e,p34m,p34l,p34u,p34_1,p34_1_1,p34_2,p34_2_1_1,p34_2_1_2,p34_2_1_3,...,p34_10_4,p34_10_5,p34_10_6,p34_10_7,p34_10_8,p34_11,p34_12,p34_12_1,p34_13,p34_14
2,9,9015,90150001,2,4,1,3,2,2,2,...,2,2,2,2,2,1,2,-32,2,-33
4,21,21114,211140001,2,8,1,3,1,1,1,...,2,2,2,2,2,1,2,-32,2,-33
5,21,21114,211140001,2,8,2,3,1,1,1,...,1,1,1,1,1,1,2,-32,2,-33
6,21,21114,211140001,2,4,1,2,1,1,1,...,1,1,1,1,1,1,2,-32,2,-33
7,21,21114,211140001,2,4,1,2,1,1,1,...,1,1,1,1,1,1,2,-32,2,-33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,28,28032,280320001,2,4,1,3,1,2,2,...,2,2,2,1,2,2,2,-32,2,-33
783,28,28032,280320001,2,3,1,3,1,2,1,...,2,2,2,1,2,2,2,-32,2,-33
784,26,26002,260020001,2,3,1,3,1,2,2,...,2,2,2,1,2,2,2,-32,2,-33
790,28,99997,999999997,97,3,1,3,1,2,2,...,2,2,2,1,2,2,2,-32,2,-33


In [13]:
emifdev1.columns

Index(['folio', 'trim', 'unidad', 'ciudad', 'punto', 'turno', 'p1', 'p2',
       'p3a', 'p3n',
       ...
       'p52', 'p52_1', 'p52_2', 'p52_3', 'nivel', 'oficio_mx', 'filtro',
       'estrato', 'upm', 'ponfin3'],
      dtype='object', length=237)

In [18]:
# Filtrar las columnas que comienzan con 'p34'
columnas_p34 = [col for col in emifdev1.columns if col.startswith('p34_3')]

# Crear un nuevo dataframe solo con las columnas filtradas
emifdev1_filtrado = emifdev1[columnas_p34]

In [19]:
emifdev1_filtrado.head()

,p34_3_0_1,p34_3_0_2,p34_3_0_3,p34_3_0_4,p34_3_1_1,p34_3_1_2,p34_3_1_3,p34_3_1_4
0,2,2,2,2,-31,-31,-31,-31
1,2,2,2,2,-31,-31,-31,-31
2,2,2,1,2,2,2,1,2
3,2,2,2,2,-31,-31,-31,-31
4,1,1,1,2,2,2,1,2


### Stat 3: Organized Crime Precense Perception

    elif row['Bien jurídico afectado'] == 'Otros bienes jurídicos afectados (del fuero común)' and row['Tipo de delito'] == 'Narcomenudeo':
    elif row['Subtipo de delito'] == 'Homicidio doloso' and row['Modalidad']== 'Con arma de fuego':
    elif row['Bien jurídico afectado'] == 'Libertad personal'

    
     return 'Crimen Organizado'
    

In [127]:
#risk2023_actualizado.to_file("C:\\Users\\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\1. Mapas\Metodo_1\\areas\\municipiosrisk.shp")


In [128]:
risk2023_actualizado.head(1)

,municipio_,cvemunic,nomedo,nommun,area,corrupcion,crimenorg,violencia,vio_muj,risk2_trat,risk2_abus,risk2_esta,risk2_dels,vio_inst,geometry,criorg_nor
0,"Aguascalientes, Aguascalientes",1001,Aguascalientes,Aguascalientes,1168.762384,474.0,1734,1011.0,36.008395,28.187135,13.268622,11.241027,44.32964,37.2695,"POLYGON ((-102.14207 22.07321, -102.12842 22.0...",11.725103


In [122]:
# Primero, renombra la columna 'Cve. Municipio' en 'grupo_total' para que coincida con 'cvemunic' en 'risk2023'
grupo_total = grupo_total.rename(columns={'Cve. Municipio': 'cvemunic'})

# Luego, haz el merge de ambos dataframes
risk2023_actualizado = risk2023.merge(grupo_total[['cvemunic', 'Total']], on='cvemunic', how='left')

# Sobrescribe la columna 'crimenorg' con los valores de la columna 'total'
risk2023_actualizado['crimenorg'] = risk2023_actualizado['Total']

# Elimina la columna 'total' ya que solo se usó para actualizar 'crimenorg'
risk2023_actualizado = risk2023_actualizado.drop(columns=['Total'])

In [125]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Supongamos que 'risk2023_actualizado' es tu DataFrame actualizado con la columna 'crimenorg'
# Crea un objeto MinMaxScaler
scaler = MinMaxScaler(feature_range=(1, 100))

# Ajusta y transforma los datos de 'crimenorg'
risk2023_actualizado['criorg_nor'] = scaler.fit_transform(risk2023_actualizado[['crimenorg']])

# Verifica el resultado
print(risk2023_actualizado.head())


                       municipio_  cvemunic          nomedo          nommun  \
0  Aguascalientes, Aguascalientes      1001  Aguascalientes  Aguascalientes   
1        Asientos, Aguascalientes      1002  Aguascalientes        Asientos   
2        Calvillo, Aguascalientes      1003  Aguascalientes        Calvillo   
3           Cosio, Aguascalientes      1004  Aguascalientes           Cosio   
4        El Llano, Aguascalientes      1010  Aguascalientes        El Llano   

          area  corrupcion  crimenorg  violencia    vio_muj  risk2_trat  \
0  1168.762384       474.0       1734     1011.0  36.008395   28.187135   
1   547.762077         4.0        127       28.0   1.969570         NaN   
2   931.300088         9.0         62       27.0   1.934942         NaN   
3   128.907513         1.0         40        8.0   1.277020         NaN   
4   500.861359         7.0         18        8.0   1.277020         NaN   

   risk2_abus  risk2_esta  risk2_dels   vio_inst  \
0   13.268622   11.241

In [ ]:
risk2023 = gpd.read_file("C:\\Users\\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\1. Mapas\Metodo_1\\areas\\municipiosrisk.shp")

In [108]:
ssp = pd.read_csv("C:\\Users\\puria\\Mapa_Migracion_Irregular_Mexico\\1. Mapas\\3.datos\\2. datos_estadisticos\\3. risks\SSP\\2023.csv")

In [111]:
# Lista de los nombres de las columnas que corresponden a los meses
month_columns = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Crear la nueva columna 'Total' con la suma de los valores de los meses
ssp['Total'] = ssp[month_columns].sum(axis=1)

In [115]:
# Crear el subdataframe con las condiciones especificadas
sub_ssp = ssp[
    ((ssp['Bien jurídico afectado'] == 'Otros bienes jurídicos afectados (del fuero común)') & (ssp['Tipo de delito'] == 'Narcomenudeo')) |
    ((ssp['Subtipo de delito'] == 'Homicidio doloso') & (ssp['Modalidad'] == 'Con arma de fuego')) |
    (ssp['Bien jurídico afectado'] == 'Libertad personal')
]

# Mostrar las primeras filas del subdataframe para verificar
print(sub_ssp.head())


     Año  Clave_Ent         Entidad  Cve. Municipio       Municipio  \
0   2023          1  Aguascalientes            1001  Aguascalientes   
24  2023          1  Aguascalientes            1001  Aguascalientes   
25  2023          1  Aguascalientes            1001  Aguascalientes   
26  2023          1  Aguascalientes            1001  Aguascalientes   
27  2023          1  Aguascalientes            1001  Aguascalientes   

              Bien jurídico afectado Tipo de delito Subtipo de delito  \
0   La vida y la Integridad corporal      Homicidio  Homicidio doloso   
24                 Libertad personal      Secuestro         Secuestro   
25                 Libertad personal      Secuestro         Secuestro   
26                 Libertad personal      Secuestro         Secuestro   
27                 Libertad personal      Secuestro         Secuestro   

                         Modalidad  Enero  ...  Abril  Mayo  Junio  Julio  \
0                Con arma de fuego      1  ...      1    

In [119]:
# Asegurarse de que la columna 'total' es de tipo numérico
sub_ssp['total'] = pd.to_numeric(sub_ssp['Total'], errors='coerce')

# Agrupar por 'Cve. Municipio' y sumar los valores de 'total'
grupo_total = sub_ssp.groupby('Cve. Municipio')['Total'].sum().reset_index()

# Mostrar las primeras filas del dataframe resultante para verificar
print(grupo_total.head())


   Cve. Municipio  Total
0            1001   1734
1            1002    127
2            1003     62
3            1004     40
4            1005    266


C:\Users\puria\AppData\Local\Temp\ipykernel_11540\3502033165.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_ssp['total'] = pd.to_numeric(sub_ssp['Total'], errors='coerce')


In [129]:
grupo_total.head()

,cvemunic,Total
0,1001,1734
1,1002,127
2,1003,62
3,1004,40
4,1005,266
